In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import KFold
from sklearn.metrics import classification_report
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from transformers import BertTokenizer, BertModel, AdamW, get_linear_schedule_with_warmup
from tqdm import tqdm


/opt/conda/envs/default/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:

# 设置随机种子
seed = 42
torch.manual_seed(seed)
torch.cuda.manual_seed(seed)
np.random.seed(seed)

# 加载数据
file_path = '../datasets_FIX2/FIX2_deduplicated_mangoNews_Nums3000p_CategoryMerge_new_undersampled_Example.csv'
# file_path = '../datasets_FIX2/FIX2_deduplicated_mangoNews_Nums3000p_CategoryMerge_new_undersampled.csv'

data = pd.read_csv(file_path,low_memory=False,lineterminator="\n")

# 加载BERT tokenizer和模型
model_name = '../bert-base-multilingual-cased'
tokenizer = BertTokenizer.from_pretrained(model_name)
bert_model = BertModel.from_pretrained(model_name)

# 将模型移动到GPU
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
bert_model.to(device)



In [3]:
# 定义数据集类
class NewsDataset(Dataset):
    def __init__(self, texts, labels, tokenizer, max_length=512):
        self.texts = texts
        self.labels = labels
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, index):
        text = str(self.texts[index])
        label = self.labels[index]

        encoding = self.tokenizer.encode_plus(
            text,
            add_special_tokens=True,
            max_length=self.max_length,
            return_token_type_ids=False,
            padding='max_length',
            return_attention_mask=True,
            return_tensors='pt',
            truncation=True
        )

        return {
            'text': text,
            'input_ids': encoding['input_ids'].flatten(),
            'attention_mask': encoding['attention_mask'].flatten(),
            'label': torch.tensor(label, dtype=torch.long)
        }

# 将孟加拉语类别转换为数字标签
label_map = {label: i for i, label in enumerate(data['category1'].unique())}
labels = data['category1'].map(label_map).tolist()

# 定义LSTM分类器
class LSTMClassifier(nn.Module):
    def __init__(self, bert_model, hidden_size, num_classes, num_layers=1, bidirectional=True, dropout=0.5):
        super(LSTMClassifier, self).__init__()
        self.bert = bert_model
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.bidirectional = bidirectional
        self.dropout = nn.Dropout(dropout)
        self.lstm = nn.LSTM(bert_model.config.hidden_size, hidden_size, num_layers, bidirectional=bidirectional, batch_first=True)
        self.fc = nn.Linear(hidden_size * (2 if bidirectional else 1), num_classes)

    def forward(self, input_ids, attention_mask):
        outputs = self.bert(input_ids=input_ids, attention_mask=attention_mask)
        last_hidden_state = outputs.last_hidden_state
        lstm_output, _ = self.lstm(last_hidden_state)
        lstm_output = self.dropout(lstm_output[:, -1, :])
        logits = self.fc(lstm_output)
        return logits

# 定义训练和评估函数
def train_epoch(model, data_loader, optimizer, scheduler, device):
    model.train()
    total_loss = 0
    progress_bar = tqdm(data_loader, desc='Training', leave=False)
    for batch in progress_bar:
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['label'].to(device)

        optimizer.zero_grad()
        logits = model(input_ids, attention_mask)
        print(logits)
        print(logits.shape)
        print(labels)
        print(labels.size())
        
        loss = nn.CrossEntropyLoss()(logits, labels)
        loss.backward()
        optimizer.step()
        scheduler.step()

        total_loss += loss.item()
        progress_bar.set_postfix({'loss': loss.item()})
    return total_loss / len(data_loader)

def evaluate(model, data_loader, device, label_map):
    model.eval()
    predictions = []
    true_labels = []
    with torch.no_grad():
        for batch in data_loader:
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            labels = batch['label'].to(device)

            logits = model(input_ids, attention_mask)
            batch_predictions = torch.argmax(logits, dim=1)
            predictions.extend(batch_predictions.tolist())
            true_labels.extend(labels.tolist())

    label_map_inv = {v: k for k, v in label_map.items()}
    predictions = [label_map_inv[i] for i in predictions]
    true_labels = [label_map_inv[i] for i in true_labels]

    report = classification_report(true_labels, predictions, digits=4)
    return report



In [4]:
# 设置超参数
num_epochs = 2
batch_size =4
learning_rate = 2e-5
hidden_size = 128
num_classes = len(label_map)
kfold = KFold(n_splits=5, shuffle=True, random_state=seed)

# 存储所有fold的性能指标
all_reports = []

# K-Fold交叉验证
for fold, (train_idx, val_idx) in enumerate(kfold.split(data)):
    print(f'Fold {fold + 1}')
    
    train_data = data.iloc[train_idx]
    val_data = data.iloc[val_idx]

    train_texts = train_data['body'].tolist()
    train_labels = train_data['category1'].map(label_map).tolist()
    val_texts = val_data['body'].tolist()
    val_labels = val_data['category1'].map(label_map).tolist()

    train_dataset = NewsDataset(train_texts, train_labels, tokenizer)
    val_dataset = NewsDataset(val_texts, val_labels, tokenizer)

    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    val_loader = DataLoader(val_dataset, batch_size=batch_size)

    model = LSTMClassifier(bert_model, hidden_size, num_classes)
    model.to(device)

    optimizer = AdamW(model.parameters(), lr=learning_rate)
    total_steps = len(train_loader) * num_epochs
    scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=0, num_training_steps=total_steps)

    best_val_loss = float('inf')
    for epoch in range(num_epochs):
        train_loss = train_epoch(model, train_loader, optimizer, scheduler, device)
        val_report = evaluate(model, val_loader, device, label_map)

        print(f'Epoch {epoch + 1}/{num_epochs}')
        print(f'Train Loss: {train_loss:.4f}')
        print('Validation Report:')
        print(val_report)

        val_loss = 1 - float(val_report.split('\n')[-2].split()[-2])  # 提取验证集损失
        if val_loss < best_val_loss:
            best_val_loss = val_loss
            torch.save(model.state_dict(), f'besta_model_fold_{fold + 1}.pth')

    # 在每个fold结束后,评估最佳模型在验证集上的性能
    best_model = LSTMClassifier(bert_model, hidden_size, num_classes)
    best_model.load_state_dict(torch.load(f'besta_model_fold_{fold + 1}.pth'))
    best_model.to(device)
    val_report = evaluate(best_model, val_loader, device, label_map)
    all_reports.append(val_report)

    print(f'Fold {fold + 1} Best Validation Report:')
    print(val_report)
    print()
    



Fold 1


/opt/conda/envs/default/lib/python3.8/site-packages/transformers/optimization.py:429: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
Training:   0%|          | 0/200 [00:00<?, ?it/s]

tensor([[-0.3022,  0.0833,  0.0065,  0.1480,  0.0466,  0.0124,  0.4299,  0.2868,
         -0.1390],
        [-0.2332,  0.0940,  0.0132,  0.3554,  0.2112, -0.1020,  0.2920,  0.1797,
         -0.0688],
        [-0.0917,  0.0694,  0.0541,  0.1740,  0.0213, -0.1568,  0.1943,  0.0707,
          0.0640],
        [-0.2144, -0.0488,  0.0340,  0.1591,  0.2035, -0.1157,  0.0993,  0.0322,
          0.0656]], device='cuda:0', grad_fn=<AddmmBackward0>)
torch.Size([4, 9])
tensor([2, 3, 8, 1], device='cuda:0')
torch.Size([4])


Training:   1%|          | 2/200 [00:00<01:24,  2.33it/s, loss=2.21]

tensor([[-0.2687,  0.1010,  0.3368,  0.1046, -0.0022,  0.0440,  0.0258,  0.0593,
          0.0848],
        [-0.1179,  0.1243,  0.2572, -0.1599,  0.1176, -0.0150, -0.0968, -0.0684,
          0.3300],
        [-0.0784,  0.2030,  0.2357,  0.0205, -0.2039, -0.0344,  0.1700,  0.0121,
          0.1421],
        [-0.1622,  0.1535,  0.2404, -0.1118,  0.1789,  0.0450, -0.0150, -0.0622,
          0.0926]], device='cuda:0', grad_fn=<AddmmBackward0>)
torch.Size([4, 9])
tensor([1, 6, 1, 6], device='cuda:0')
torch.Size([4])


...

Training:  10%|▉         | 19/200 [00:05<00:44,  4.11it/s, loss=2.26]

tensor([[-0.3602,  0.3585,  0.1841, -0.0130, -0.2256,  0.0487,  0.1710, -0.2745,
         -0.0035],
        [-0.1484,  0.7172,  0.1484,  0.2972, -0.2441,  0.2913,  0.3247, -0.1673,
         -0.3037],
        [-0.1658,  0.3051,  0.0714,  0.2631, -0.2827,  0.2755,  0.2113, -0.1202,
         -0.2443],
        [-0.0643,  0.5013,  0.0598,  0.2971, -0.0166,  0.4549,  0.1943, -0.0339,
         -0.0042]], device='cuda:0', grad_fn=<AddmmBackward0>)
torch.Size([4, 9])
tensor([8, 3, 7, 4], device='cuda:0')
torch.Size([4])
tensor([[-0.2302,  0.2968,  0.0109,  0.0689,  0.2242,  0.0865,  0.2129, -0.1457,
          0.0824],
        [-0.4363,  0.3731,  0.0483,  0.1068,  0.0414,  0.1624,  0.2452, -0.2163,
          0.0848],
        [-0.2452,  0.6634,  0.2592,  0.0864, -0.0946,  0.2095,  0.4221, -0.1153,
         -0.0099],
        [ 0.0015,  0.5014,  0.3063,  0.1471, -0.2404,  0.5820,  0.0873, -0.3201,
         -0.0947]], device='cuda:0', grad_fn=<AddmmBackward0>)
torch.Size([4, 9])
tensor([6, 4, 7, 8],

Training:  10%|█         | 21/200 [00:06<00:49,  3.59it/s, loss=2.38]

tensor([[-0.2578,  0.4973,  0.2203,  0.3343, -0.2428,  0.1570,  0.1546, -0.0083,
         -0.2099],
        [-0.1109,  0.3372,  0.0208,  0.2043, -0.1945,  0.3846,  0.1782, -0.0648,
         -0.1484],
        [-0.0906,  0.7579,  0.1916,  0.2716, -0.2421,  0.4335,  0.3112, -0.2567,
          0.0317],
        [-0.2120,  0.5395,  0.2111,  0.2034, -0.4685,  0.5433,  0.2333, -0.0853,
         -0.0630]], device='cuda:0', grad_fn=<AddmmBackward0>)
torch.Size([4, 9])
tensor([3, 7, 7, 0], device='cuda:0')
torch.Size([4])


KeyboardInterrupt: 

In [ ]:
# 计算并打印所有fold的平均性能
print('Average Performance Across All Folds:')
avg_report = pd.DataFrame([report.split() for report in all_reports]).mean(axis=0)
avg_report = '\n'.join([('{:<10}'.format(col) + '{:.4f}'.format(val)) for col, val in zip(avg_report.index, avg_report.values)])
print(avg_report)

In [9]:
print(all_reports)

['              precision    recall  f1-score   support\n\n    অন্যান্য     1.0000    0.7222    0.8387        18\n    অর্থনীতি     0.9630    0.8966    0.9286        29\n         আইন     0.8667    0.9630    0.9123        27\n    খেলাধুলা     0.9583    1.0000    0.9787        23\n     বিজ্ঞান     0.9200    0.9583    0.9388        24\n      বিনোদন     0.9545    1.0000    0.9767        21\n     রাজনীতি     0.9615    1.0000    0.9804        25\n  লাইফস্টাইল     0.9412    0.9412    0.9412        17\n      শিক্ষা     0.8750    0.8750    0.8750        16\n\n    accuracy                         0.9350       200\n   macro avg     0.9378    0.9285    0.9300       200\nweighted avg     0.9377    0.9350    0.9336       200\n', '              precision    recall  f1-score   support\n\n    অন্যান্য     1.0000    0.8696    0.9302        23\n    অর্থনীতি     1.0000    0.9000    0.9474        20\n         আইন     1.0000    1.0000    1.0000        22\n    খেলাধুলা     1.0000    1.0000    1.0000        25

In [11]:
# 计算并打印所有fold的平均性能
print('Average Performance Across All Folds:')
all_lines = [report.split('\n') for report in all_reports]
header = all_lines[0][0] + '\t' + '\t'.join([line.strip() for line in all_lines[0][-4:]])

values = []
for report in all_reports:
    lines = report.split('\n')
    cls_lines = lines[1:-5]
    cls_values = []
    for line in cls_lines:
        parts = line.split()
        if len(parts) >= 5:
            cls_values.append([float(val) if val != 'nan' else 0.0 for val in parts[1:-1]])
    values.append(cls_values)

avg_values = np.mean(values, axis=0)
cls_report = '\n'.join([f'{cls}\t{prec:.4f}\t{rec:.4f}\t{f1:.4f}\t{sup:.0f}' for cls, (prec, rec, f1, sup) in zip(label_map.keys(), avg_values)])

avg_acc = np.mean([float(line.split()[-2]) for report in all_reports for line in report.split('\n') if 'accuracy' in line])
avg_macro = np.mean([float(line.split()[-2]) for report in all_reports for line in report.split('\n') if 'macro avg' in line])
avg_weighted = np.mean([float(line.split()[-2]) for report in all_reports for line in report.split('\n') if 'weighted avg' in line])

avg_report = header + '\n' + cls_report + '\n' + f"accuracy\t{avg_acc:.4f}\nmacro avg\t{avg_macro:.4f}\nweighted avg\t{avg_weighted:.4f}"
print(avg_report)

Average Performance Across All Folds:


ValueError: not enough values to unpack (expected 4, got 3)